In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, kruskal, ttest_ind, t, kstwo, wilcoxon
from utils_best_arm import calc_return
from tqdm import tqdm
import numpy as np
import seaborn as sns


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
symbols = os.listdir("./Results/")
path1 = f"./Results/{symbols[0]}/exp2_concat/"
rewards = os.listdir(path1)
indexes_df = []

bh_df = []
for symbol in symbols[:]:
    path1 = f"./Results/{symbol}/exp2_concat/{rewards[0]}"
    df = pd.read_csv(path1).fillna(0)

    bh_percentage = df.iloc[:, 0].values
    test_portfolios = {0: df.iloc[:, 0].values}
    indexes = calc_return(bh_percentage, test_portfolios)
    bh_df.append(indexes)

bh_df = pd.concat(bh_df, axis=1).T
bh_df.columns = [f"B&H_{col.replace('_test', '')}" for col in bh_df.columns]
bh_df = pd.DataFrame(np.tile(bh_df, [100, 1]), columns=bh_df.columns)
bh_df.reset_index(drop=True, inplace=True)



for reward in tqdm(rewards[:]):
    ls2 = []
    for symbol in symbols:
        path1 = f"./Results/{symbol}/exp2_concat/{reward}"
        df = pd.read_csv(path1).fillna(0)

        bh_percentage = df.iloc[:, 0].values
        seeds = list(df.columns)[1:]
        ls = []
        for seed in seeds:
            test_portfolios = {seed: df.loc[:, seed].values}
            indexes = calc_return(bh_percentage, test_portfolios)
            ls.append(indexes)

        ls_df = pd.concat(ls, axis=1).T
        ls_df.columns = [f"{reward.replace('_0.csv', '')}_{col}" for col in ls_df.columns]
        ls2.append(ls_df)
    
    ls2 = pd.concat(ls2)
    indexes_df.append(ls2)


indexes_df = pd.concat(indexes_df, axis=1)
indexes_df.reset_index(drop=True, inplace=True)
indexes_df = pd.concat([bh_df, indexes_df], axis=1)
indexes_df.to_csv("./ts-run-results/[exp2]data-for-wilcoxon-test.csv", index=False)
indexes_df

100%|██████████| 5/5 [04:34<00:00, 54.84s/it]


,profit_sortino_test,profit_sharpe_test,profit_risk_test,profit_mdd_test,profit_downrisk_test,profit_cumreturn_test,regularized_0.01.csv_sortino_test,regularized_0.01.csv_sharpe_test,regularized_0.01.csv_risk_test,regularized_0.01.csv_mdd_test,...,regularized_0.1.csv_risk_test,regularized_0.1.csv_mdd_test,regularized_0.1.csv_downrisk_test,regularized_0.1.csv_cumreturn_test,regularized_0.2.csv_sortino_test,regularized_0.2.csv_sharpe_test,regularized_0.2.csv_risk_test,regularized_0.2.csv_mdd_test,regularized_0.2.csv_downrisk_test,regularized_0.2.csv_cumreturn_test
0,3.871673,2.448116,0.221965,0.124035,0.140352,0.679396,4.158359,2.607396,0.219285,0.124035,...,0.022363,0.026231,0.118404,-0.026231,0.000000,-0.737545,0.006670,0.006490,0.000000,-0.004929
1,3.152395,2.094754,0.237061,0.189043,0.157526,0.597161,4.017667,2.543916,0.221228,0.124035,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2.380883,1.637781,0.248816,0.227177,0.171158,0.457045,3.759913,2.510647,0.238013,0.123706,...,0.172991,0.046245,0.082037,0.626212,5.043650,2.652663,0.186136,0.069031,0.097897,0.610018
3,2.910500,1.947060,0.232029,0.189043,0.155222,0.529085,1.027587,0.755655,0.271958,0.290114,...,0.153361,0.041006,0.071344,0.420319,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3.021448,2.001666,0.233776,0.189043,0.154873,0.553345,4.171997,2.610492,0.218489,0.106812,...,0.164103,0.041006,0.070910,0.573684,-0.171728,-0.809192,0.025517,0.032278,0.120237,-0.020759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5.719206,3.295122,0.163183,0.073345,0.094018,0.688655,5.523589,3.142959,0.159085,0.073345,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
996,2.460955,1.854947,0.198992,0.178580,0.149990,0.417609,4.599494,2.929710,0.165075,0.082197,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
997,4.285209,2.711073,0.174954,0.099822,0.110686,0.581931,2.715000,1.604847,0.131852,0.130272,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
998,4.102009,2.585474,0.174047,0.099822,0.109701,0.544237,5.507937,3.182617,0.150864,0.062174,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:

def replace_reward(name):
    if "regularized" in name:
        return f"FPR-{name.split('_')[1].split('.csv')[0]}_{name.split('_')[2]}"
    elif "old_sharpe" in name:
        return f"PSR_{name.split('_')[-1]}"
    elif "old_profit" in name:
        return f"PP1_{name.split('_')[-1]}"
    elif "profit" == name.split("_")[0]:
        return f"FP5_{name.split('_')[-1]}"
    else:
        return name


dt = {
    "profit_0": "FP5",
    "regularized_0.01": "FPR-0.01",
    "regularized_0.05": "FPR-0.05",
    "regularized_0.1": "FPR-0.1",
    "regularized_0.2": "FPR-0.2",
}

replace_name = lambda x: dt[x.replace(".csv", "")]

cols = [col.replace("_test", "") for col in indexes_df.columns]
indexes_df.columns = list(map(replace_reward, cols))
indexes_df

,B&H_sortino,B&H_sharpe,B&H_risk,B&H_mdd,B&H_downrisk,B&H_cumreturn,FP5_sortino,FP5_sharpe,FP5_risk,FP5_mdd,...,FPR-0.1_risk,FPR-0.1_mdd,FPR-0.1_downrisk,FPR-0.1_cumreturn,FPR-0.2_sortino,FPR-0.2_sharpe,FPR-0.2_risk,FPR-0.2_mdd,FPR-0.2_downrisk,FPR-0.2_cumreturn
0,-0.157004,-0.112971,0.285712,0.365095,0.205582,-0.070543,3.871673,2.448116,0.221965,0.124035,...,0.022363,0.026231,0.118404,-0.026231,0.000000,-0.737545,0.006670,0.006490,0.000000,-0.004929
1,0.836190,0.643763,0.236477,0.155024,0.182059,0.132149,3.152395,2.094754,0.237061,0.189043,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.034455,0.025707,0.240779,0.207193,0.179644,-0.022560,2.380883,1.637781,0.248816,0.227177,...,0.172991,0.046245,0.082037,0.626212,5.043650,2.652663,0.186136,0.069031,0.097897,0.610018
3,0.732692,0.484028,0.312351,0.246703,0.206344,0.107767,2.910500,1.947060,0.232029,0.189043,...,0.153361,0.041006,0.071344,0.420319,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.573555,-0.420436,0.324648,0.331021,0.237978,-0.172746,3.021448,2.001666,0.233776,0.189043,...,0.164103,0.041006,0.070910,0.573684,-0.171728,-0.809192,0.025517,0.032278,0.120237,-0.020759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.308166,0.970799,0.338935,0.247834,0.251526,0.311847,5.719206,3.295122,0.163183,0.073345,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
996,0.922114,0.659294,0.263219,0.174759,0.188196,0.148917,2.460955,1.854947,0.198992,0.178580,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
997,-0.531712,-0.384728,0.240386,0.214923,0.173935,-0.114382,4.285209,2.711073,0.174954,0.099822,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
998,0.118557,0.085178,0.205550,0.149467,0.147679,-0.003678,4.102009,2.585474,0.174047,0.099822,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
rewards2 = list(map(replace_name, rewards))

criteria = ["sortino", "sharpe", "risk", "mdd", "downrisk", "cumreturn",]
test_df = []
for c in criteria:
    cols = [col for col in indexes_df.columns if c == col.split("_")[-1]]

    tmp_df = indexes_df[cols]

    n = tmp_df.shape[0] * 2 - 2

    vars = list(tmp_df.columns)

    for var1 in vars:
        if var1 in rewards: continue
        
        for _var in vars:
            if _var == var1:
                continue

            test = wilcoxon(tmp_df[var1], tmp_df[_var])

            _var1 = var1.split("_")[0]
            _var2 = _var.split("_")[-1]

            median1 = round(tmp_df[var1].median(), 4)
            median2 = round(tmp_df[_var].median(), 4)

            if _var2 in ["cumreturn", "sortino", "sharpe"]:
                if median1 > median2:
                    sign = 1
                else:
                    sign = 0
            else:
                if median1 < median2:
                    sign = 1
                else:
                    sign = 0
        

            test_df.append({
                "reward1": _var1,
                "var": _var2,
                "reward2": f"{_var.split('_')[0]}",
                "statistic": test.statistic,
                "pvalue": round(test.pvalue, 4),
                "median1": median1,
                "median2": median2,
                "median1 > median2": sign,
            })


test_df = pd.DataFrame(test_df)
test_df = test_df.sort_values("reward1").reset_index(drop=True)
test_df["var"] = test_df["var"].replace({
    "cumreturn": "CR",
    "downrisk": "DR",
    "mdd": "MDD",
    "risk": "Risk",
    "sharpe": "SR",
    "sortino": "STR",
})


test_df.to_csv("./ts-run-results/[exp2]wilcoxon-test.csv", index=False)
test_df

,reward1,var,reward2,statistic,pvalue,median1,median2,median1 > median2
0,B&H,STR,FP5,22403.0,0.000,0.0765,2.5886,0
1,B&H,SR,FPR-0.05,44299.0,0.000,0.0554,1.9851,0
2,B&H,SR,FPR-0.1,152518.0,0.000,0.0554,0.0000,1
3,B&H,SR,FPR-0.2,231215.0,0.037,0.0554,0.0000,1
4,B&H,Risk,FP5,112570.0,0.000,0.2593,0.2235,0
...,...,...,...,...,...,...,...,...
175,FPR-0.2,DR,B&H,159.0,0.000,0.0000,0.1878,1
176,FPR-0.2,DR,FP5,2049.0,0.000,0.0000,0.1519,1
177,FPR-0.2,DR,FPR-0.01,2707.0,0.000,0.0000,0.1380,1
178,FPR-0.2,DR,FPR-0.1,4171.5,0.000,0.0000,0.0000,0


In [3]:
test_df = pd.read_csv("./ts-run-results/[exp2]wilcoxon-test.csv",)

In [4]:
def convert_sig(x):
    if x < 0.01:
        return "***"
    elif 0.01 <= x < 0.05:
        return "**"
    elif 0.05 <= x < 0.1:
        return "*"
    else:
        return ""


def convert_star(x):
    if x == 1:
        return "√" 
    elif np.isnan(x):
        return "-"
    else:
        return ""


def construct_CR(var):
    print(var)
    tmp_df = test_df[test_df["var"] == var]
    CR_df1 = pd.pivot_table(tmp_df, values="statistic", index="reward1", columns="reward2").round(0).astype(str).applymap(lambda x: "-" if x == "nan" else x)

    CR_df2 = pd.pivot_table(tmp_df, values="pvalue", index="reward1", columns="reward2").round(2)
    CR_df3 = CR_df2.applymap(convert_sig)
    CR_df2 = CR_df2.astype(str)
    CR_df4 = pd.pivot_table(tmp_df, values="median1 > median2", index="reward1", columns="reward2")
    CR_df4 = CR_df4.applymap(convert_star)
    CR_df = CR_df1 + "^{" + CR_df3 + "}(" + CR_df4 + ")"
    CR_df = CR_df.applymap(lambda x: x.replace("()", ""))
    CR_df = CR_df.applymap(lambda x: "-" if "-^" in x else x)
    
    CR_df.index.name = "reward"
    CR_df.to_csv(f"./ts-run-results/[exp2]wilcoxon-test-criteria-{var}.csv")

    return CR_df


vars = test_df["var"].unique().tolist()
for var in vars:
    CR_df = construct_CR(var)


STR
SR
Risk
CR
MDD
DR


In [5]:
var = vars[1]
CR_df = construct_CR(var)
CR_df

SR


reward2,B&H,FP5,FPR-0.01,FPR-0.05,FPR-0.1,FPR-0.2
reward,,,,,,
B&H,-,23164.0^{***},16125.0^{***},44299.0^{***},152518.0^{***}(√),231215.0^{**}(√)
FP5,23164.0^{***}(√),-,173245.0^{***},236522.0^{},101188.0^{***}(√),22853.0^{***}(√)
FPR-0.01,16125.0^{***}(√),173245.0^{***}(√),-,206282.0^{***}(√),74473.0^{***}(√),15049.0^{***}(√)
FPR-0.05,44299.0^{***}(√),236522.0^{}(√),206282.0^{***},-,36456.0^{***}(√),4834.0^{***}(√)
FPR-0.1,152518.0^{***},101188.0^{***},74473.0^{***},36456.0^{***},-,4678.0^{***}
FPR-0.2,231215.0^{**},22853.0^{***},15049.0^{***},4834.0^{***},4678.0^{***},-
